In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
os.environ["HF_HOME"] = "/raid/user_eduardoaugusto/hf_cache"

from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "tiiuae/Falcon-H1-0.5B-Base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

prompt = "Hello, how are you?"

inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(**inputs, max_new_tokens=100)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The fast path for FalconH1 will be used when running the model on a GPU


Hello, how are you?')
    print('Hello, how are you?')
    print('Hello, how are you?')
    print('Hello, how are you?')
    print('Hello, how are you?')
    print('Hello, how are you?')
    print('Hello, how are you?')
    print('Hello, how are you?')
    print('Hello, how are you?')
    print('Hello, how are you?')


In [6]:
import os
models = set()
with open("../llm_pt_leaderboard_raw_results/lfs-file-list.txt", "r") as f:
    for line in f:
        if 'max_gen_toks' in line:
            print(line)
            models.add("/".join(line.split()[-1].split("/")[:2]))

3e8f8788e4 - BornSaint/Dare_Angel_8B/raw_2025-07-07T06-21-20.661894/pretrained__BornSaint__Dare_Angel_8B,dtype__float16,device__cuda:0,revision__main,trust_remote_code__True,max_gen_toks__1536,starting_max_length__2560_assin2_rte.jsonl

4eda3b6079 - BornSaint/Dare_Angel_8B/raw_2025-07-07T06-21-20.661894/pretrained__BornSaint__Dare_Angel_8B,dtype__float16,device__cuda:0,revision__main,trust_remote_code__True,max_gen_toks__1536,starting_max_length__2560_assin2_sts.jsonl

36c63b45fc - BornSaint/Dare_Angel_8B/raw_2025-07-07T06-21-20.661894/pretrained__BornSaint__Dare_Angel_8B,dtype__float16,device__cuda:0,revision__main,trust_remote_code__True,max_gen_toks__1536,starting_max_length__2560_bluex.jsonl

4c1c7c0f32 - BornSaint/Dare_Angel_8B/raw_2025-07-07T06-21-20.661894/pretrained__BornSaint__Dare_Angel_8B,dtype__float16,device__cuda:0,revision__main,trust_remote_code__True,max_gen_toks__1536,starting_max_length__2560_enem_challenge.jsonl

e0c6316f28 - BornSaint/Dare_Angel_8B/raw_2025-07-07T0

In [9]:
models

{'BornSaint/Dare_Angel_8B',
 'CEIA-UFG/Gemma-3-Gaia-PT-BR-4b-it',
 'HuggingFaceTB/SmolLM3-3B',
 'HuggingFaceTB/SmolLM3-3B-Base',
 'Qwen/Qwen2.5-14B',
 'Qwen/Qwen2.5-14B-Instruct',
 'THUDM/GLM-4-32B-0414',
 'THUDM/GLM-Z1-32B-0414',
 'THUDM/GLM-Z1-Rumination-32B-0414',
 'TIGER-Lab/Qwen2.5-32B-Instruct-CFT',
 'TheDrummer/Big-Tiger-Gemma-27B-v1',
 'YOYO-AI/Qwen2.5-32B-YOYO-reasoning',
 'allenai/OLMo-2-0325-32B',
 'allenai/OLMo-2-0325-32B-Instruct',
 'byroneverson/gemma-2-27b-it-abliterated',
 'deepcogito/cogito-v1-preview-llama-70B',
 'deepseek-ai/DeepSeek-R1-Distill-Llama-8B',
 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B',
 'deepseek-ai/DeepSeek-R1-Distill-Qwen-32B',
 'google/gemma-3n-E2B',
 'google/gemma-3n-E2B-it',
 'google/gemma-3n-E4B',
 'google/gemma-3n-E4B-it',
 'google/medgemma-27b-it',
 'google/medgemma-27b-text-it',
 'google/medgemma-4b-it',
 'google/medgemma-4b-pt',
 'huihui-ai/DeepSeek-R1-Distill-Qwen-14B-abliterated-v2',
 'langtech-languagemodeling/IberianLLM-7B-Instruct',
 'mi

In [11]:
with open("affected_models.txt", "w") as f:
    for model in models:
        f.write(model+'\n')

In [16]:
import os
import glob
import json
for filepath in glob.glob("../llm_pt_leaderboard_requests/**/*.json", recursive=True):
    with open(filepath, "r") as f:
        data = json.load(f)
        #print(data)
    if data['model'] in models and data['status'] not in ['RUNNING']:
        if 'lm_eval_model_type' in data and data['lm_eval_model_type'] != 'huggingface':
            data['lm_eval_model_type'] = 'huggingface'
        if 'error_msg' in data or 'traceback' in data:
            del data['error_msg']
            del data['traceback']
        data['status'] = 'PENDING_NEW_EVAL'
        with open(filepath, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)